<a href="https://colab.research.google.com/github/mukulnag09/grammer-checker-using-bert/blob/main/grammar_checker%20for%20csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
!pip install transformers==3.5.1
!pip install torch==1.4.0
!pip install wget
import csv

from transformers import BertForSequenceClassification

output_dir = '/content/drive/MyDrive/model_save'
from transformers import BertTokenizer
import torch
# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained(output_dir,local_files_only=True)
model_loaded = BertForSequenceClassification.from_pretrained(output_dir)

Loading BERT tokenizer...


In [49]:
# Let's check it for a given sentence
import csv
from csv import writer
from csv import reader
from google.colab import files
k=[]
x=0
with open('/content/drive/MyDrive/model_save/read.csv') as file_obj:
  reader_obj = csv.reader(file_obj)
  for row in reader_obj:
    sent = str(row)
    x+=1
    encoded_dict = tokenizer.encode_plus(
                            sent,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = 64,           # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                      )
    
        # Add the encoded sentence to the list.    
    input_id = encoded_dict['input_ids']
    
        # And its attention mask (simply differentiates padding from non-padding).
    attention_mask = encoded_dict['attention_mask']
    input_id = torch.LongTensor(input_id)
    attention_mask = torch.LongTensor(attention_mask)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model_loaded = model_loaded.to(device)
    input_id = input_id.to(device)
    attention_mask = attention_mask.to(device)
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model_loaded(input_id, token_type_ids=None, attention_mask=attention_mask)

    logits = outputs[0]
    index = logits.argmax()

    if index == 1:
      k.append(1)
    else:
      k.append(0)


with open('/content/drive/MyDrive/model_save/read.csv','r') as f , open('/content/drive/MyDrive/model_save/test_data.csv', 'a', newline='') as g:
    i=0
    w = writer(g)
    r = reader(f)
    
    for row in r:
      
      if i==0:
        row.append("label")
        w.writerow(row)

      else:
        row.append(k[i])
        w.writerow(row)

      i+=1




/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
